#### 1. Preparation

In [ ]:
# Packages
import numpy as np

import geopandas as gpd
import shapely.geometry
import json

import pathlib

In [ ]:
# Necessary variables
resolution_func = 5
chunk_size_func = 100
processor_func = 4

h_crs = 2193
v_crs = 7839

# Direct the basepath to the folder
# that contains two files
# one is 'rec2_3.geojson'
# other one is 'flow_and_friction.csv.gz'
basepath = pathlib.Path(r"S:\Grid_Ori_bigdata")
lidar_dataset_name = "Wellington_2013"
result_folder = "dem_generation"

#### 2. Create neccesary files

In [ ]:
data_dir = basepath / result_folder
data_dir.mkdir(exist_ok=True)

In [ ]:
# Padding box coordinates
padding_func = [1767790, 5471400, 1774510, 5475880]

# Bounding box - Part 1
x0 = padding_func[0]
y0 = padding_func[1]
x1 = padding_func[2]
y1 = padding_func[3]
catchment = shapely.geometry.Polygon([(x0, y0), (x1, y0), (x1, y1), (x0, y1)])
catchment = gpd.GeoSeries([catchment])
catchment = catchment.set_crs(h_crs)

# Write out to file
catchment.to_file(basepath / result_folder / "catchment_boundary.geojson", crs="EPSG:2193", driver="GeoJSON")

#### 3. Setup for instructions

In [ ]:
instruction_json = {}

In [ ]:
# DRAIN
instruction_json['drains'] = {
        # drain - output
        "output": {
            "crs": {
                "horizontal": h_crs,
                "vertical": v_crs
            },
            "grid_params": {
                "resolution": 1
            }
        },
    
        # drain - processing
        "processing": {
            "chunk_size": 1000,
            "number_of_cores": 4
        },
    
        # drain - data_paths
        "data_paths": {
            "local_cache": str(basepath),
            "subfolder": result_folder,
            "catchment_boundary": "catchment_boundary.geojson"
        },
      
        # drain - apis
        "apis": {
            "open_topography": {
                "Wellington_2013": {
                    "crs": {
                        "horizontal": h_crs,
                        "vertical": v_crs
                    }
                }
            },
            "linz": {
                "key": "857413f41ce446ed8961e2f1e960a24b",
                "land": {
                    "layers": [51559]
                }
            }
        },
      
        # drain - general
        "general": {
            "lidar_classifications_to_keep": [2, 9]
        },
      
        # drain - drains
        "drains": {
            "width": 5
        }
      
    }

In [ ]:
# RIVER
instruction_json['rivers'] = {
        # river - output
        "output": {
            "crs": {
                "horizontal": h_crs,
                "vertical": v_crs
            },
            "grid_params":{
                "resolution": 1
            }
        },
        
        # river - processing
        "processing": {
            "chunk_size": 1000,
            "number_of_cores": 4
        },
        
        # river - data_paths
        "data_paths": {
            "local_cache": str(basepath),
            "subfolder": result_folder,
        },
        
        # river - apis
        "apis": {
            "open_topography": {
                "Wellington_2013": {
                    "crs": {
                        "horizontal": h_crs,
                        "vertical": v_crs
                    }
                }
            },
            "linz": {
                "key": "857413f41ce446ed8961e2f1e960a24b",
                "land": {
                    "layers": [51559]                                              # SHOULD I ROTATE?
                },
                "bathymetry_contours": {
                    "layers": [50554]                                              # SHOULD I ROTATE?
                }
            }
        },
        
        # river - general
        "general": {
            "set_dem_shoreline": True,
            "bathymetry_contours_z_label": "valdco",
            "drop_offshore_lidar": True,
            "lidar_classifications_to_keep": [2, 9],
            "interpolate_missing_values": False
        },
        
        # river - rivers
        "rivers": {
            "osm_id": 132793862,
            "veg_lidar_classifications_to_keep": [2, 3, 4, 5, 9],
            
            # Can be varied according to the formula
            "max_channel_width": 120,
            "min_channel_width": 10,
            "max_bank_height": 2,
            "min_bank_height": 0.75,
            
            "rec_file": str(basepath / "rec2_3.geojson"),
            "flow_file": str(basepath / "flow_and_friction.csv.gz"),
            
            # Vary later
            "rec_alignment_tolerance": 65,
            "width_centre_smoothing": 10,
            "channel_area_threshold": 125000000,
            "channel_rec_id": 9253579,
            "cross_section_spacing": 10,
        }
}

In [ ]:
# DEM
instruction_json['dem'] = {
    # outputs
    "output": {
        "crs": {
            "horizontal": h_crs,
            "vertical": v_crs
        },
        "grid_params":{
            "resolution": resolution_func
        }
    },
    
    # processing
    "processing": {
        "chunk_size": chunk_size_func,
        "number_of_cores": processor_func
    },
    
    # apis
    "apis": {
        "open_topography": {
            f"{lidar_dataset_name}": {
                "crs": {
                    "horizontal": h_crs,
                    "vertical": v_crs
                }
            }
        },
        "linz": {
            "key": "857413f41ce446ed8961e2f1e960a24b",
            "land": {
                "layers": [51559]                                                        
            },
            "bathymetry_contours": {                                 # ADDED
                "layers": [50554]                                    # ADDED
            }
        }
    },
    
    # data paths
    "data_paths": {
        "local_cache": str(basepath),
        "subfolder": fr"{result_folder}",
        "catchment_boundary": "catchment_boundary.geojson",
        "raw_dem": "raw_dem.nc", 
        # For bathymetry
        "river_bathymetry": ["river_bathymetry.geojson", "fan_bathymetry.geojson",
                             "open_drain_elevation_5m_width.geojson", "closed_drain_elevation_5m_width.geojson"], # ADDED
        "river_polygons": ["river_polygon.geojson", "fan_polygon.geojson",
                           "open_drain_polygon_5m_width.geojson", "closed_drain_polygon_5m_width.geojson"], # ADDED
        # Result
        "result_dem": "dem.nc"
    },
    
    # general
    "general": {
        # For lidar
        "set_dem_shoreline": True,
        "drop_offshore_lidar": False,
        "lidar_classification_to_keep": [2],
        "interpolation_method": "nearest",
        "lidar_interpolation_method": "idw",
        
        # For bathymetry
        "bathymetry_contours_z_label": "valdco",                                        
        "bathymetry_points_type": ['rivers', 'rivers', 'drains', 'drains'],   # ADDED          
        "bathymetry_points_z_label": ['bed_elevation_Rupp_and_Smart', "depths", "elevation", "elevation"] # ADDED
    }
}

In [ ]:
# Save out instructions
with open(basepath / result_folder / "instructions.json", "w") as instruction:
    json.dump(instruction_json, instruction)